In [1]:
# Please visit:
#
# https://download.nkg-mn.com/credo/anomalies/
#
# Prepared dots_100000.pickle and others can be downloaded from mentioned website.
from keras.layers import RandomFlip, RandomRotation
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from scipy.ndimage import rotate




# metryka - za
# czy nie ma warstw inne niż proste konwolucje (np. falkowe, np. warstwy od transformacji hougha)
# poeównianie kulbacka leubrera (VAE)

In [2]:
from dataset_loader import load_dataset_with_augmentation, save_to_file, load_from_file, cut_image_to_size

DOTS_SRC = 'hit-images-final2/dot'
DOTS_DST = 'cache/dots_v4.pickle'

TRACKS_SRC = 'hit-images-final2/line'
TRACKS_DST = 'cache/tracks_v4.pickle'

WORMS_SRC = 'hit-images-final2/worms'
WORMS_DST = 'cache/worms_v4.pickle'

ARTIFACTS_SRC = 'hit-images-final2/artefact'
ARTIFACTS_DST = 'cache/artifacts_v4.pickle'

In [5]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line
from skimage import data

import matplotlib.pyplot as plt
from matplotlib import cm

from skimage.transform import probabilistic_hough_line

from dataset_loader import load_images

vs = load_images(TRACKS_SRC)
image = vs[0]

edges = canny(image, 2, 1, 25)
lines = probabilistic_hough_line(image, threshold=10, line_length=5,
                                 line_gap=3)

# Generating figure 2
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges, cmap=cm.gray)
ax[1].set_title('Canny edges')

ax[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()


In [ ]:
from dataset_loader import load_images


def make_augmentations(images, mul=1):
    if mul == 1:
        return images

    arr = []
    new_images = []
    for i, image in enumerate(images):
        #im = Image.fromarray((image * 255).astype(np.uint8))
        mask = np.where(image == 0, 0.0, 1.0)
        new_images.append(cut_image_to_size(image, 64))
        for flip in [False, True]:
            for j in range(1, mul):
                deg = 360/mul * j

                augmented_image = rotate(image, deg, reshape=False)
                augmented_mask = rotate(mask, deg, reshape=False)
                if flip:
                    augmented_image = np.flipud(augmented_image)
                    augmented_mask = np.flipud(augmented_mask)
                #augmented_image -= augmented_image.min()
                augmented_image = np.where(augmented_mask < 0.25, 0, augmented_image)
                augmented_image = np.where(augmented_image < 0, 0, augmented_image)
                augmented_image = np.where(augmented_image > 1, 1, augmented_image)
                #ai_max = augmented_image.max()
                #if ai_max > 1:
                    #augmented_image *= 1/ai_max
                arr.append(cut_image_to_size(augmented_image, 64))

    augs = np.vstack([arr])

    return new_images, augs

save_to_file(DOTS_DST, make_augmentations(load_images(DOTS_SRC, 4), 12))
save_to_file(TRACKS_DST, make_augmentations(load_images(TRACKS_SRC, 4), 12))
save_to_file(WORMS_DST, make_augmentations(load_images(WORMS_SRC, 4), 12))
save_to_file(ARTIFACTS_DST, make_augmentations(load_images(ARTIFACTS_SRC, 4), 12))

print('done')